In [1]:
# Dependencies
import pandas as pd
import requests
import time
import json
import operator as op
import numpy as np
import hvplot.pandas
import matplotlib
import holoviews as hv
import selenium
import bokeh.plotting
import ipywidgets
import webbrowser
import gmaps
import geoviews
import dataframe_image as dfi
from ipywidgets.embed import embed_minimal_html
from config import api_key
from scipy.stats import linregress
from matplotlib import pyplot as plt
from bokeh.plotting import figure
from ipywidgets.embed import embed_minimal_html

In [2]:
# Import cities file as DataFrame
data_df = pd.read_csv("athlete_events.csv")
data_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
# checking columns of dataframe
data_df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [4]:
# Unique cities that held Olympic games
cities_held_games = data_df['City'].unique()
cities_held_games

array(['Barcelona', 'London', 'Antwerpen', 'Paris', 'Calgary',
       'Albertville', 'Lillehammer', 'Los Angeles', 'Salt Lake City',
       'Helsinki', 'Lake Placid', 'Sydney', 'Atlanta', 'Stockholm',
       'Sochi', 'Nagano', 'Torino', 'Beijing', 'Rio de Janeiro', 'Athina',
       'Squaw Valley', 'Innsbruck', 'Sarajevo', 'Mexico City', 'Munich',
       'Seoul', 'Berlin', 'Oslo', "Cortina d'Ampezzo", 'Melbourne',
       'Roma', 'Amsterdam', 'Montreal', 'Moskva', 'Tokyo', 'Vancouver',
       'Grenoble', 'Sapporo', 'Chamonix', 'St. Louis', 'Sankt Moritz',
       'Garmisch-Partenkirchen'], dtype=object)

In [5]:
cities_coordinates = pd.read_csv("olym.csv")
cities_coordinates.head()

,City,Country,NOC,Summer,Winter,Year,Latitude,Longitude
0,Athens,Greece,GRE,I,NaN,1896,37.983333,23.733334
1,Paris,France,FRA,II,NaN,1900,48.853410,2.348800
2,St. Louis,United States,USA,III,NaN,1904,38.627273,-90.197891
3,London,United Kingdom,GBR,IV,NaN,1908,51.508415,-0.125533
4,Stockholm,Sweden,SWE,V,NaN,1912,59.332577,18.064903


In [6]:
# Configure the map plot
plot_1 = cities_coordinates.hvplot.points(
    'Longitude',
    'Latitude',
    geo=True, 
    tiles='OSM',  # Use OSM as the tile source
    frame_width=700, 
    frame_height=500, 
    color='City',
    alpha=0.5,
    title='Host Cities from 1896 - 2016',
    scale=0.8)

# Save the Figure using Holoviews
#plot_1.save("output_data/Host_Cities_Map.png")

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City)

In [7]:
# Reading host cities both seasons CSV
all_host_cities = pd.read_csv("host_cities_both_seasons.csv")
all_host_cities.head()

,year,host_city,country,continent,season
0,1896,Athens,Greece,Europe,Summer
1,1900,Paris,France,Europe,Summer
2,1904,St. Louis,United States,North America,Summer
3,1908,London,United Kingdom,Europe,Summer
4,1912,Stockholm,Sweden,Europe,Summer


In [8]:
# Unique cities that held Olympic games
cities_held_games = all_host_cities['continent'].unique()
cities_held_games

array(['Europe', 'North America', 'Oceania', 'Asia', 'Central America',
       'South America'], dtype=object)

In [9]:
all_host_cities_df = all_host_cities['continent'].value_counts().reset_index()
all_host_cities_df.columns = ['Continent', 'Count of Games held 1896 - 2016']

all_host_cities_df = pd.DataFrame(all_host_cities_df)
all_host_cities_df

,Continent,Count of Games held 1896 - 2016
0,Europe,30
1,North America,11
2,Asia,5
3,Oceania,2
4,Central America,1
5,South America,1


In [25]:
styled_all_host_cities_df = all_host_cities_df.style.bar(subset=["Count of Games held 1896 - 2016"], color="lightblue", vmin=0)
styled_all_host_cities_df

,Continent,Count of Games held 1896 - 2016
0,Europe,30
1,North America,11
2,Asia,5
3,Oceania,2
4,Central America,1
5,South America,1
